In [4]:
#Import pandas library
import pandas as pd

In [8]:
#Load file 
school_data = "Resources/schools_complete.csv"
student_data = "Resources/students_complete.csv"

In [9]:
#Merge into single file
school_merge = pd.merge(student_data, school_data, how = "left", on=["school_name","school_name"])

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed